In [1]:
!pip -q install "transformers>=4.42" "datasets>=2.20" "evaluate>=0.4.2" "scikit-learn>=1.5" "accelerate>=0.30" "tqdm>=4.66"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00


In [2]:
import sys, numpy, torch, transformers, datasets, sklearn
print(sys.version)
print("numpy", numpy.__version__)
print("torch", torch.__version__)
print("transformers", transformers.__version__)
print("datasets", datasets.__version__)


3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
numpy 2.0.2
torch 2.8.0+cu126
transformers 4.56.1
datasets 4.0.0


# Clone the dataset

In [3]:
REPO_URL = "https://github.com/Dimasss16/nlp-offensive-language-classifier.git" # some url...
BRANCH   = "main"
VERIFY_DATA = True

import os, subprocess, shutil, csv
from pathlib import Path

%cd /content
repo_dir = REPO_URL.rsplit("/", 1)[-1].replace(".git", "")
if Path(repo_dir).exists():
    print(f"Repo '{repo_dir}' exists — pulling latest…")
    %cd /content/{repo_dir}
    if BRANCH:
        subprocess.run(["git", "fetch", "origin", BRANCH], check=True)
        subprocess.run(["git", "checkout", BRANCH], check=True)
        subprocess.run(["git", "reset", "--hard", f"origin/{BRANCH}"], check=True)
    else:
        subprocess.run(["git", "pull"], check=True)
else:
    print("Cloning the repo")
    subprocess.run(["git", "clone", REPO_URL], check=True)
    %cd /content/{repo_dir}
    if BRANCH:
        subprocess.run(["git", "checkout", BRANCH], check=True)

print("Repo root:", Path.cwd())

def short_tree(root=".", max_depth=2):
    root = Path(root)
    for p in sorted(root.rglob("*")):
        depth = len(p.relative_to(root).parts)
        if depth <= max_depth:
            print("  " * (depth-1) + ("├─ " if depth>0 else "") + p.name)

short_tree(".", max_depth=2)


if VERIFY_DATA:
    expected = [
        "src/data/processed_bert/train.csv",
        "src/data/processed_bert/train_augmented.csv",
        "src/data/processed_bert/val.csv",
        "src/data/processed_bert/test.csv",
    ]
    missing = []
    for rel in expected:
        p = Path(rel)
        if p.exists():
            print(f"{rel} ({p.stat().st_size/1e6:.2f} MB) exists")

    def peek_csv(path, n=2):
        path = Path(path)
        try:
            with path.open(newline="", encoding="utf-8") as f:
                reader = csv.DictReader(f)
                headers = reader.fieldnames or []
                print(f"\n{path} headers:", headers)
                required = {"text", "clean_text", "label"}
                lowered = {h.lower() for h in headers}
                print("Columns OK." if required.issubset(lowered)
                      else f"Missing columns: {sorted(required - lowered)}")
                for _, row in zip(range(n), reader):
                    row_show = {k: (v[:80]+"…") if isinstance(v, str) and len(v) > 80 else v
                                for k, v in row.items()}
                    print(row_show)
        except Exception as e:
            print(f"Peek failed for {path}: {e}")

    for rel in expected:
        if Path(rel).exists():
            peek_csv(rel)


/content
Cloning the repo
/content/nlp-offensive-language-classifier
Repo root: /content/nlp-offensive-language-classifier
├─ .DS_Store
├─ .git
  ├─ HEAD
  ├─ branches
  ├─ config
  ├─ description
  ├─ hooks
  ├─ index
  ├─ info
  ├─ logs
  ├─ objects
  ├─ packed-refs
  ├─ refs
├─ .gitignore
├─ .idea
  ├─ inspectionProfiles
  ├─ libraries
  ├─ misc.xml
  ├─ modules.xml
  ├─ nlp-offensive-language-classifier.iml
  ├─ prettier.xml
  ├─ vcs.xml
  ├─ workspace.xml
├─ .python-version
├─ LICENSE
├─ README.md
├─ models
  ├─ .DS_Store
  ├─ .gitkeep
  ├─ bert_augmented
  ├─ bert_original
  ├─ logreg
  ├─ majority
├─ notebooks
  ├─ bert_training.ipynb
  ├─ bert_training_fixed.ipynb
  ├─ perturbation_tests.ipynb
  ├─ stage1_evaluation.ipynb
  ├─ {01_eda.ipynb}
├─ predictions
  ├─ .DS_Store
  ├─ bert
  ├─ logreg
  ├─ majority
├─ requirements.txt
├─ results
  ├─ stage1_confusion_matrices.png
  ├─ stage1_overall_metrics.csv
  ├─ stage1_per_class_comparison.png
├─ run_phase1.py
├─ src
  ├─ .DS_Store


In [4]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path
import json

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"[setup] Device: {device}")
if device.type == 'cuda':
    print(f"[setup] here is the GPU: {torch.cuda.get_device_name(0)}")

[setup] Device: cuda
[setup] here is the GPU: Tesla T4


# Config

In [5]:
BASE_DIR = Path('src/data')
MODELS_DIR = Path('models')
RESULTS_DIR = Path('results')

VAL_PATH = BASE_DIR / 'processed_bert/val.csv'
TEST_PATH = BASE_DIR / 'processed_bert/test.csv'

CONFIG = {
    'model_name': 'bert-base-uncased',
    'max_length': 128,
    'num_labels': 3,
    'seed': 42,
}

CLASS_NAMES = {0: 'hate_speech', 1: 'offensive', 2: 'neither'}

np.random.seed(CONFIG['seed'])
torch.manual_seed(CONFIG['seed'])

# Data loading

In [6]:
def load_train_data(use_augmented=False):
    if use_augmented:
        path = BASE_DIR / 'processed_bert/train_augmented.csv'
        print(f"Loading AUGMENTED training data")
    else:
        path = BASE_DIR / 'processed_bert/train.csv'
        print(f"Loading ORIGINAL training data")

    df = pd.read_csv(path)
    print(f"Loaded {len(df):,} samples from {path.name}")
    return df

In [7]:
train_orig = load_train_data(use_augmented=False)
train_aug = load_train_data(use_augmented=True)
val_df = pd.read_csv(VAL_PATH)
test_df = pd.read_csv(TEST_PATH)

Loading ORIGINAL training data
Loaded 17,347 samples from train.csv
Loading AUGMENTED training data
Loaded 22,263 samples from train_augmented.csv


In [8]:
print(f"Val: {len(val_df):,} samples")

Val: 3,718 samples


In [9]:
print(f"Test: {len(test_df):,} samples")

Test: 3,718 samples


In [10]:
def peek_data(df, name="Dataset"):
    print(f"\n[peek] {name}")
    print(f"  Shape: {df.shape}")
    print(f"  Columns: {list(df.columns)}")

    dist = df['label'].value_counts().sort_index()
    print(f"  Classes:")
    for label, count in dist.items():
        pct = (count / len(df)) * 100
        print(f"    {CLASS_NAMES[label]:12s}: {count:5d} ({pct:5.1f}%)")


peek_data(train_orig, "Original Training")
peek_data(train_aug, "Augmented Training")
peek_data(val_df, "Validation")


[peek] Original Training
  Shape: (17347, 3)
  Columns: ['text', 'label', 'clean_text']
  Classes:
    hate_speech :  1001 (  5.8%)
    offensive   : 13432 ( 77.4%)
    neither     :  2914 ( 16.8%)

[peek] Augmented Training
  Shape: (22263, 3)
  Columns: ['text', 'label', 'clean_text']
  Classes:
    hate_speech :  3003 ( 13.5%)
    offensive   : 13432 ( 60.3%)
    neither     :  5828 ( 26.2%)

[peek] Validation
  Shape: (3718, 3)
  Columns: ['text', 'label', 'clean_text']
  Classes:
    hate_speech :   215 (  5.8%)
    offensive   :  2879 ( 77.4%)
    neither     :   624 ( 16.8%)


# Class Weights Calculation

In [31]:
def calculate_class_weights(df):
    from sklearn.utils.class_weight import compute_class_weight

    labels = df['label'].values
    unique_labels = np.array([0, 1, 2])

    weights = compute_class_weight(
        class_weight='balanced',
        classes=unique_labels,
        y=labels
    )

    print(f"[weights] Computed for {len(df):,} samples:")
    for label, weight in zip(unique_labels, weights):
        print(f"  {CLASS_NAMES[label]:12s}: {weight:.4f}")

    return torch.tensor(weights, dtype=torch.float32)

In [32]:
weights_orig = calculate_class_weights(train_orig)
weights_aug = calculate_class_weights(train_aug)

[weights] Computed for 17,347 samples:
  hate_speech : 5.7766
  offensive   : 0.4305
  neither     : 1.9843
[weights] Computed for 22,263 samples:
  hate_speech : 2.4712
  offensive   : 0.5525
  neither     : 1.2733


# Tokenization

In [12]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [13]:
from datasets import Dataset

train_orig_dataset = Dataset.from_pandas(train_orig[['clean_text', 'label']])
print(f"Original train: {len(train_orig_dataset):,} samples")

train_aug_dataset = Dataset.from_pandas(train_aug[['clean_text', 'label']])
print(f"Augmented train: {len(train_aug_dataset):,} samples")

val_dataset = Dataset.from_pandas(val_df[['clean_text', 'label']])
print(f"Validation: {len(val_dataset):,} samples")

Original train: 17,347 samples
Augmented train: 22,263 samples
Validation: 3,718 samples


In [14]:
def tokenize_texts(examples):
    return tokenizer(
        examples['clean_text'],
        padding='max_length',
        truncation=True,
        max_length=128
    )

In [15]:
train_orig_tok = train_orig_dataset.map(
    tokenize_texts,
    batched=True,
    desc="Tokenizing original"
)

Tokenizing original:   0%|          | 0/17347 [00:00<?, ? examples/s]

In [16]:
train_aug_tok = train_aug_dataset.map(
    tokenize_texts,
    batched=True,
    desc="Tokenizing augmented"
)

Tokenizing augmented:   0%|          | 0/22263 [00:00<?, ? examples/s]

In [17]:
val_tok = val_dataset.map(
    tokenize_texts,
    batched=True,
    desc="Tokenizing validation"
)

Tokenizing validation:   0%|          | 0/3718 [00:00<?, ? examples/s]

In [18]:
train_orig_tok = train_orig_tok.rename_column('label', 'labels')
train_aug_tok = train_aug_tok.rename_column('label', 'labels')
val_tok = val_tok.rename_column('label', 'labels')

train_orig_tok.set_format('torch')
train_aug_tok.set_format('torch')
val_tok.set_format('torch')

In [19]:
sample = train_orig_tok[0]

print(f"Label: {sample['labels']} ({CLASS_NAMES[int(sample['labels'])]})")
print(f"Input IDs shape: {sample['input_ids'].shape}")
print(f"Attention mask shape: {sample['attention_mask'].shape}")

decoded = tokenizer.decode(sample['input_ids'], skip_special_tokens=True)
print(f"Decoded text (first 100 chars): '{decoded[:100]}...'")

num_real_tokens = sample['attention_mask'].sum().item()
print(f"Real tokens: {num_real_tokens} / 128")

Label: 1 (offensive)
Input IDs shape: torch.Size([128])
Attention mask shape: torch.Size([128])
Decoded text (first 100 chars): 'lmfaoooooo gay as fuck rt @ user get these two faggots off my tl [ url ]...'
Real tokens: 28 / 128


# Setting up the training

In [20]:
def get_training_config(use_augmented=False):
    train_config = {
        'num_train_epochs': 4,
        'learning_rate': 3e-5,
        'per_device_train_batch_size': 16,
        'per_device_eval_batch_size': 32,
        'gradient_accumulation_steps': 2,
        'weight_decay': 0.01,
        'warmup_ratio': 0.1,
        'fp16': True,
        'seed': CONFIG['seed'],
        'eval_strategy': 'epoch',
        'save_strategy': 'epoch',
        'save_total_limit': 2,
        'load_best_model_at_end': True,
        'metric_for_best_model': 'f1_macro',
        'greater_is_better': True,
        'logging_steps': 100,
    }

    if use_augmented:
        train_config['output_dir'] = str(MODELS_DIR / 'bert_augmented')
        train_config['run_name'] = 'bert_augmented'
    else:
        train_config['output_dir'] = str(MODELS_DIR / 'bert_original')
        train_config['run_name'] = 'bert_original'

    print(f"[config] Training config for {train_config['run_name']}")
    print(f"  Output: {train_config['output_dir']}")
    print(f"  Epochs: {train_config['num_train_epochs']}")
    print(f"  Effective batch: {train_config['per_device_train_batch_size'] * train_config['gradient_accumulation_steps']}")

    return train_config


In [21]:
config_orig = get_training_config(use_augmented=False)
config_aug = get_training_config(use_augmented=True)

[config] Training config for bert_original
  Output: models/bert_original
  Epochs: 4
  Effective batch: 32
[config] Training config for bert_augmented
  Output: models/bert_augmented
  Epochs: 4
  Effective batch: 32


In [22]:
config_orig

{'num_train_epochs': 4,
 'learning_rate': 3e-05,
 'per_device_train_batch_size': 16,
 'per_device_eval_batch_size': 32,
 'gradient_accumulation_steps': 2,
 'weight_decay': 0.01,
 'warmup_ratio': 0.1,
 'fp16': True,
 'seed': 42,
 'eval_strategy': 'epoch',
 'save_strategy': 'epoch',
 'save_total_limit': 2,
 'load_best_model_at_end': True,
 'metric_for_best_model': 'f1_macro',
 'greater_is_better': True,
 'logging_steps': 100,
 'output_dir': 'models/bert_original',
 'run_name': 'bert_original'}

In [23]:
config_aug

{'num_train_epochs': 4,
 'learning_rate': 3e-05,
 'per_device_train_batch_size': 16,
 'per_device_eval_batch_size': 32,
 'gradient_accumulation_steps': 2,
 'weight_decay': 0.01,
 'warmup_ratio': 0.1,
 'fp16': True,
 'seed': 42,
 'eval_strategy': 'epoch',
 'save_strategy': 'epoch',
 'save_total_limit': 2,
 'load_best_model_at_end': True,
 'metric_for_best_model': 'f1_macro',
 'greater_is_better': True,
 'logging_steps': 100,
 'output_dir': 'models/bert_augmented',
 'run_name': 'bert_augmented'}

# Define Metrics Function

In [24]:
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predictions)
    macro_f1 = f1_score(labels, predictions, average='macro')
    weighted_f1 = f1_score(labels, predictions, average='weighted')

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions,
        average=None,
        labels=[0, 1, 2]
    )

    return {
        'accuracy': accuracy,
        'f1_macro': macro_f1,
        'f1_weighted': weighted_f1,
        'hate_speech_precision': precision[0],
        'hate_speech_recall': recall[0],
        'hate_speech_f1': f1[0],
        'offensive_precision': precision[1],
        'offensive_recall': recall[1],
        'offensive_f1': f1[1],
        'neither_precision': precision[2],
        'neither_recall': recall[2],
        'neither_f1': f1[2],
    }

# Custom Trainer with Weighted Loss

In [25]:
from transformers import Trainer
from torch import nn

class WeightedTrainer(Trainer):
    def __init__(self, class_weights, **kwargs):
        if 'tokenizer' in kwargs:
            kwargs['processing_class'] = kwargs.pop('tokenizer')
        super().__init__(**kwargs)
        self.loss_fn = nn.CrossEntropyLoss(weight=class_weights.to(self.args.device))

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        loss = self.loss_fn(outputs.logits.view(-1, 3), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Train Function

# Training original model

In [26]:
print(f"{train_orig_tok.column_names}")

['clean_text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask']


The tokenized datasets still contain the original clean_text column, which is a string and can't be converted to tensors for training. That's why we are going to go ahead and remove them and then double check that our columns are in proper format

In [27]:
train_orig_tok = train_orig_tok.remove_columns(['clean_text'])
train_aug_tok = train_aug_tok.remove_columns(['clean_text'])
val_tok = val_tok.remove_columns(['clean_text'])

In [28]:
train_orig_tok.set_format('torch')
train_aug_tok.set_format('torch')
val_tok.set_format('torch')

In [29]:
from transformers import BertForSequenceClassification, TrainingArguments, EarlyStoppingCallback
import json
from pathlib import Path
import warnings

# We know that this is gonna happen so we'll just suppress it
warnings.filterwarnings('ignore', message='Some weights of BertForSequenceClassification were not initialized')

def train_bert(use_augmented=False):
    if use_augmented:
        print("\n" + "="*60)
        print("TRAINING ON AUGMENTED DATA")
        print("="*60)
        train_dataset = train_aug_tok
        class_weights = weights_aug
        config = config_aug
    else:
        print("\n" + "="*60)
        print("TRAINING ON ORIGINAL DATA")
        print("="*60)
        train_dataset = train_orig_tok
        class_weights = weights_orig
        config = config_orig

    print(f"[setup] Training samples: {len(train_dataset):,}")
    print(f"[setup] Validation samples: {len(val_tok):,}")
    print(f"[setup] Output directory: {config['output_dir']}")

    output_dir = Path(config['output_dir'])
    output_dir.mkdir(parents=True, exist_ok=True)

    print("\n[model] Loading BERT-base-uncased...")
    model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased',
        num_labels=3,
        ignore_mismatched_sizes=True
    )
    print(f"[model] Parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"[model] Classifier layer initialized randomly (expected)")

    training_config = {k: v for k, v in config.items() if k != 'run_name'}

    training_args = TrainingArguments(
        **training_config,
        report_to='none',
        push_to_hub=False,
        remove_unused_columns=True,
        dataloader_drop_last=False,
        dataloader_num_workers=2,
    )

    steps_per_epoch = len(train_dataset) // (config['per_device_train_batch_size'] * config['gradient_accumulation_steps'])
    total_steps = steps_per_epoch * config['num_train_epochs']

    print(f"\n[train] Training configuration:")
    print(f"  Epochs: {config['num_train_epochs']}")
    print(f"  Steps per epoch: {steps_per_epoch}")
    print(f"  Total steps: {total_steps}")
    print(f"  Effective batch size: {config['per_device_train_batch_size'] * config['gradient_accumulation_steps']}")
    print(f"  Learning rate: {config['learning_rate']}")

    print(f"\n[train] Using class weights: {class_weights.numpy()}")

    trainer = WeightedTrainer(
        class_weights=class_weights,
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_tok,
        processing_class=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
    )

    # Training
    print("\n[train] Starting training")
    print("-" * 60)

    try:
        train_result = trainer.train()
    except Exception as e:
        print(f"[ERROR] Training failed: {e}")
        return None

    print("-" * 60)
    print(f"\n[train] Training complete!")
    print(f"[train] Final loss: {train_result.training_loss:.4f}")

    # Saving everything
    best_model_dir = output_dir / 'best_model'
    print(f"\n[save] Saving model to {best_model_dir}")

    # Save model and tokenizer
    trainer.save_model(best_model_dir)
    tokenizer.save_pretrained(best_model_dir)

    # Save training history
    history_file = output_dir / 'training_history.json'

    log_history = []
    for entry in trainer.state.log_history:
        clean_entry = {}
        for k, v in entry.items():
            if hasattr(v, 'item'):  # numpy scalar
                clean_entry[k] = v.item()
            elif isinstance(v, (list, dict, str, int, float, bool, type(None))):
                clean_entry[k] = v
            else:
                clean_entry[k] = str(v)
        log_history.append(clean_entry)

    history = {
        'loss_history': log_history,
        'best_metric': float(trainer.state.best_metric) if trainer.state.best_metric else None,
        'best_model_checkpoint': trainer.state.best_model_checkpoint,
        'total_steps': int(trainer.state.global_step),
        'epochs_trained': float(trainer.state.epoch),
    }

    with open(history_file, 'w') as f:
        json.dump(history, f, indent=2)
    print(f"[save] Training history saved")

    # Save class weights
    weights_file = output_dir / 'class_weights.json'
    weights_dict = {str(i): float(w) for i, w in enumerate(class_weights.numpy())}
    with open(weights_file, 'w') as f:
        json.dump(weights_dict, f, indent=2)
    print(f"[save] Class weights saved")

    # Save config
    config_file = output_dir / 'training_config.json'
    with open(config_file, 'w') as f:
        json.dump(config, f, indent=2)
    print(f"[save] Training config saved")

    # Save dataset info
    dataset_info = {
        'train_samples': len(train_dataset),
        'val_samples': len(val_tok),
        'augmented': use_augmented,
        'dataset_name': 'augmented' if use_augmented else 'original'
    }
    info_file = output_dir / 'dataset_info.json'
    with open(info_file, 'w') as f:
        json.dump(dataset_info, f, indent=2)
    print(f"[save] Dataset info saved")

    print("\n" + "="*60)
    print("TRAINING COMPLETE")
    print("="*60)
    print(f"Model saved to: {output_dir}")

    if trainer.state.best_metric:
        print(f"\nBest validation F1: {trainer.state.best_metric:.4f}")
        print(f"Training stopped at epoch: {trainer.state.epoch:.1f}")

    return str(output_dir)

In [33]:
model_dir_original = train_bert(use_augmented=False)
print(f"Original model saved to: {model_dir_original}")


TRAINING ON ORIGINAL DATA
[setup] Training samples: 17,347
[setup] Validation samples: 3,718
[setup] Output directory: models/bert_original

[model] Loading BERT-base-uncased...


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[model] Parameters: 109,484,547
[model] Classifier layer initialized randomly (expected)

[train] Training configuration:
  Epochs: 4
  Steps per epoch: 542
  Total steps: 2168
  Effective batch size: 32
  Learning rate: 3e-05

[train] Using class weights: [5.776557  0.4304894 1.9843285]

[train] Starting training
------------------------------------------------------------


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted,Hate Speech Precision,Hate Speech Recall,Hate Speech F1,Offensive Precision,Offensive Recall,Offensive F1,Neither Precision,Neither Recall,Neither F1
1,0.522900,0.516304,0.805810,0.703112,0.845183,0.222510,0.800000,0.348178,0.978852,0.787774,0.872979,0.885350,0.891026,0.888179
2,0.378300,0.515860,0.890802,0.758863,0.900333,0.358543,0.595349,0.447552,0.966667,0.906565,0.935652,0.868381,0.919872,0.893385
3,0.333800,0.566657,0.887036,0.752030,0.896496,0.346591,0.567442,0.430335,0.962949,0.902744,0.931875,0.865067,0.924679,0.893881
4,0.195300,0.721985,0.899139,0.763742,0.904591,0.392617,0.544186,0.456140,0.955524,0.925321,0.940180,0.889241,0.900641,0.894904


------------------------------------------------------------

[train] Training complete!
[train] Final loss: 0.3971

[save] Saving model to models/bert_original/best_model
[save] Training history saved
[save] Class weights saved
[save] Training config saved
[save] Dataset info saved

TRAINING COMPLETE
Model saved to: models/bert_original

Best validation F1: 0.7637
Training stopped at epoch: 4.0

Original model saved to: models/bert_original


# Training augmented model

In [34]:
model_dir_augmented = train_bert(use_augmented=True)
print(f"[complete] Augmented model saved to: {model_dir_augmented}")


TRAINING ON AUGMENTED DATA
[setup] Training samples: 22,263
[setup] Validation samples: 3,718
[setup] Output directory: models/bert_augmented

[model] Loading BERT-base-uncased...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[model] Parameters: 109,484,547
[model] Classifier layer initialized randomly (expected)

[train] Training configuration:
  Epochs: 4
  Steps per epoch: 695
  Total steps: 2780
  Effective batch size: 32
  Learning rate: 3e-05

[train] Using class weights: [2.4711955 0.5524866 1.2733356]

[train] Starting training
------------------------------------------------------------


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted,Hate Speech Precision,Hate Speech Recall,Hate Speech F1,Offensive Precision,Offensive Recall,Offensive F1,Neither Precision,Neither Recall,Neither F1
1,0.405600,0.382379,0.893760,0.758213,0.900388,0.387622,0.553488,0.455939,0.968889,0.908649,0.937802,0.827004,0.942308,0.880899
2,0.245900,0.529157,0.901022,0.760615,0.904155,0.409091,0.502326,0.450939,0.946906,0.935394,0.941115,0.900000,0.879808,0.889789
3,0.135500,0.675929,0.906939,0.753195,0.905829,0.440000,0.409302,0.424096,0.941420,0.948941,0.945165,0.896104,0.884615,0.890323
4,0.050600,0.846072,0.903712,0.750012,0.903380,0.422535,0.418605,0.420561,0.938747,0.947551,0.943129,0.904841,0.868590,0.886345


------------------------------------------------------------

[train] Training complete!
[train] Final loss: 0.2646

[save] Saving model to models/bert_augmented/best_model
[save] Training history saved
[save] Class weights saved
[save] Training config saved
[save] Dataset info saved

TRAINING COMPLETE
Model saved to: models/bert_augmented

Best validation F1: 0.7606
Training stopped at epoch: 4.0

[complete] Augmented model saved to: models/bert_augmented


No more retraining so i'll just save the artifacts

In [35]:
from google.colab import drive

In [36]:
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
!cp -r models /content/drive/MyDrive/nlp_offensive_backup/

In [39]:
!ls -lh /content/drive/MyDrive/nlp_offensive_backup/bert_*/best_model/*.safetensors

-rw------- 1 root root 418M Sep 13 19:47 /content/drive/MyDrive/nlp_offensive_backup/bert_augmented/best_model/model.safetensors
-rw------- 1 root root 418M Sep 13 19:48 /content/drive/MyDrive/nlp_offensive_backup/bert_original/best_model/model.safetensors


# Eval and perturbation

In [40]:
from transformers import (
BertTokenizer,
BertForSequenceClassification,
Trainer,
TrainingArguments
)
from sklearn.metrics import accuracy_score, f1_score,precision_recall_fscore_support
from datasets import Dataset
import pandas as pd
import numpy as np
import torch

In [48]:
BASE_DIR = Path('src/data')
MODELS_DIR = Path('models')
RESULTS_DIR = Path('results')

TEST_CSV = BASE_DIR / 'processed_bert/test.csv'
OUT_ROOT = Path('results')

TEST_PATH_CLEAN = BASE_DIR / "processed_bert/test.csv"
TEST_PATH_PERTURBED = BASE_DIR / "processed_bert/test_perturbation.csv"

MODELS_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

TEST_PATHS = {"clean": TEST_PATH_CLEAN, "perturbed": TEST_PATH_PERTURBED}

MODELS  = ["bert_original", "bert_augmented"]
SPLITS  = ["clean", "perturbed"]

In [42]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    macro_f1 = f1_score(labels, predictions, average='macro')
    weighted_f1 = f1_score(labels, predictions, average='weighted')
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=None, labels=[0,1,2]
    )
    return {
        'accuracy': acc,
        'f1_macro': macro_f1,
        'f1_weighted': weighted_f1,
        'hate_speech_precision': precision[0], 'hate_speech_recall': recall[0], 'hate_speech_f1': f1[0],
        'offensive_precision': precision[1],   'offensive_recall': recall[1],   'offensive_f1': f1[1],
        'neither_precision': precision[2],     'neither_recall': recall[2],     'neither_f1': f1[2],
    }

In [43]:
def tokenize_dataset(df, tokenizer, desc="perturbed test"):
    dataset = Dataset.from_pandas(df[['clean_text','label']])
    def tokenize_batch(ex):
        return tokenizer(ex['clean_text'], padding='max_length', truncation=True, max_length=128)
    tokenized = dataset.map(tokenize_batch, batched=True, desc=f"Tokenizing {desc}")
    tokenized = tokenized.rename_column('label','labels')
    tokenized = tokenized.remove_columns(['clean_text'])
    tokenized.set_format('torch')
    return tokenized

In [49]:
def eval(model_name: str, split: str = "clean"):
    test_path = TEST_PATHS[split]
    test_df = pd.read_csv(test_path)

    model_dir  = MODELS_DIR / f"{model_name}" / "best_model"

    output_dir = RESULTS_DIR / f"{model_name}" / f"{split}"
    output_dir.mkdir(parents=True, exist_ok=True)

    model     = BertForSequenceClassification.from_pretrained(model_dir)
    tokenizer = BertTokenizer.from_pretrained(model_dir)

    args = TrainingArguments(output_dir=str(output_dir),
                             per_device_eval_batch_size=32,
                             report_to='none')
    trainer = Trainer(model=model, tokenizer=tokenizer,
                      args=args, compute_metrics=compute_metrics)

    test_tok = tokenize_dataset(test_df, tokenizer, desc=f"{split} test")

    print(f"[{model_name}] Predicting on {split} test…")
    test_output = trainer.predict(test_tok)
    test_preds  = np.argmax(test_output.predictions, axis=1)
    test_probs  = torch.softmax(torch.tensor(test_output.predictions), dim=-1).numpy()

    pd.DataFrame({
        'true_label':      test_df['label'].values,
        'predicted_label': test_preds,
        'prob_hate':       test_probs[:, 0],
        'prob_offensive':  test_probs[:, 1],
        'prob_neither':    test_probs[:, 2],
    }).to_csv(f"{output_dir}/test_predictions.csv", index=False)

    pd.DataFrame([test_output.metrics]).to_csv(f"{output_dir}/metrics.csv", index=False)

    print(f"[{model_name}] Saved to {output_dir}/")
    return test_output.metrics

In [50]:
metrics_orig_clean = eval("bert_original",  split="clean")
metrics_aug_clean  = eval("bert_augmented", split="clean")

/tmp/ipython-input-2810358293.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, tokenizer=tokenizer,


Tokenizing clean test:   0%|          | 0/3718 [00:00<?, ? examples/s]

[bert_original] Predicting on clean test…


[bert_original] Saved to results/bert_original/clean/


/tmp/ipython-input-2810358293.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, tokenizer=tokenizer,


Tokenizing clean test:   0%|          | 0/3718 [00:00<?, ? examples/s]

[bert_augmented] Predicting on clean test…


[bert_augmented] Saved to results/bert_augmented/clean/


In [51]:
metrics_orig_pert  = eval("bert_original",  split="perturbed")
metrics_aug_pert   = eval("bert_augmented", split="perturbed")

/tmp/ipython-input-2810358293.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, tokenizer=tokenizer,


Tokenizing perturbed test:   0%|          | 0/3718 [00:00<?, ? examples/s]

[bert_original] Predicting on perturbed test…


[bert_original] Saved to results/bert_original/perturbed/


/tmp/ipython-input-2810358293.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, tokenizer=tokenizer,


Tokenizing perturbed test:   0%|          | 0/3718 [00:00<?, ? examples/s]

[bert_augmented] Predicting on perturbed test…


[bert_augmented] Saved to results/bert_augmented/perturbed/


In [55]:
rows = []
for m in MODELS:
    for s in SPLITS:
        p = OUT_ROOT / m / s / "metrics.csv"
        if not p.exists():
            print(f"[missing] {p}")
            continue
        d = pd.read_csv(p).iloc[0].to_dict()
        d = {k.replace("test_", "").replace("eval_", ""): v for k, v in d.items()}
        d.update({"model": m, "split": s})
        rows.append(d)

metrics_all = pd.DataFrame(rows)

ordered = ["model","split","accuracy","f1_macro","f1_weighted",
           "hate_speech_precision","hate_speech_recall","hate_speech_f1",
           "offensive_precision","offensive_recall","offensive_f1",
           "neither_precision","neither_recall","neither_f1",
           "loss","runtime","samples_per_second","steps_per_second"]
metrics_all = metrics_all[[c for c in ordered if c in metrics_all.columns] +
                          [c for c in metrics_all.columns if c not in ordered]]

display(metrics_all.sort_values(["model","split"]).reset_index(drop=True))

if {"model","split","f1_macro"} <= set(metrics_all.columns):
    pivot = metrics_all.pivot(index="model", columns="split", values="f1_macro")
    if "clean" in pivot and "perturbed" in pivot:
        pivot["delta_f1_macro"] = pivot["perturbed"] - pivot["clean"]
        display(pivot)


,model,split,accuracy,f1_macro,f1_weighted,hate_speech_precision,hate_speech_recall,hate_speech_f1,offensive_precision,offensive_recall,offensive_f1,neither_precision,neither_recall,neither_f1,loss,runtime,samples_per_second,steps_per_second,model_preparation_time
0,bert_augmented,clean,0.910974,0.782666,0.912823,0.473684,0.546729,0.507592,0.950350,0.944078,0.947203,0.903437,0.8832,0.893204,0.284007,6.0313,616.449,19.399,0.0042
1,bert_augmented,perturbed,0.900215,0.767382,0.904056,0.423913,0.546729,0.477551,0.949982,0.930184,0.939979,0.886035,0.8832,0.884615,0.346168,6.0985,609.658,19.185,0.0029
2,bert_original,clean,0.901829,0.771690,0.906804,0.413559,0.570093,0.479371,0.953331,0.929489,0.941259,0.900974,0.8880,0.894440,0.315610,6.1768,601.926,18.942,0.0049
3,bert_original,perturbed,0.898063,0.766707,0.902973,0.412969,0.565421,0.477318,0.952126,0.925669,0.938711,0.883387,0.8848,0.884093,0.326284,6.0829,611.218,19.234,0.0028


split,clean,perturbed,delta_f1_macro
model,,,
bert_augmented,0.782666,0.767382,-0.015285
bert_original,0.771690,0.766707,-0.004983
